## Bibliotecas


In [37]:
%matplotlib inline
# importar biblioteca Pandas
import pandas as pd

# importar Matplotlib biblioteca de gráficos
import matplotlib.pyplot as plt

# importar Seaborn biblioteca de gráficos
import seaborn as sns

import geopandas as gpd

# Importar Numpy, train_test_split
import numpy as np
from sklearn.model_selection import train_test_split

# importa o modelo de regressão linear, metrics, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import metrics

from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import LabelEncoder

# acesso ao so e ao sys
import os
import sys

# Garbage collector, para recuperar memória após carregar o df inicial
import gc

import graphviz.backend as be
import statistics as sts

# Importar algoritmo
# from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from mpl_toolkits.mplot3d import axes3d

# import dtreeviz

random_state = 1234

# controlar as notificações
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)
pd.set_option("display.float_format", lambda x: "%.8f" % x)

%config InlineBackend.figure_format = 'retina' # Make visualizations look good
# %config InlineBackend.figure_format = 'svg'
%matplotlib inline

## Acessado os dados


In [38]:
url = "./faccionados_analise_vinculo.csv"

In [39]:
df = pd.read_csv(url, encoding="utf8", low_memory=False)

In [40]:
print(list(df.columns))
len(df.columns)

['Unnamed: 0', 'nome_completo', 'vulgo_alcunha', 'faccao', 'faccao_funcao', 'bairro_atual', 'cidade_atual', 'uf_atual', 'area_atuacao']


9

Tornar todos os NaN como 0 = não informado


In [41]:
df.fillna("NI", inplace=True)

In [42]:
df.head(5)

,Unnamed: 0,nome_completo,vulgo_alcunha,faccao,faccao_funcao,bairro_atual,cidade_atual,uf_atual,area_atuacao
0,1,KAYO NASCIMENTO DE MAGALHÃES,CABRA/ 99,NI,FINANCEIRO,NI,NI,NI,NI
1,2,JOSÉ MARIANO DOS SANTOS,RATO,NI,BRAÇO DIREITO,NI,NI,NI,NI
2,3,GIVALDO BARBOSA DE FRANÇA,QUINZINHO,NI,CHEFE/LÍDER,NI,NI,NI,NI
3,4,JOSÉ ERIVALDO DA PAZ,ÍNDIO,NI,CHEFE/LÍDER,NI,NI,NI,NI
4,5,LUCAS MANOEL DA SILVA,PROJETO,CV,SOLDADO DO TRÁFICO,NI,NI,NI,CLIMA BOM/FRONTEIRA


## Facções


In [43]:
print(df.groupby("faccao").size())

faccao
CV         75
NEUTRO     11
NI        107
PCC        33
SF          2
dtype: int64


In [44]:
df_grouped = df.groupby("faccao").size().reset_index(name="Contagem")
df_grouped = df_grouped.drop("Contagem", axis=1)
df_grouped["faccao_id"] = range(1, len(df_grouped) + 1)  # IDs sequenciais
df_grouped.to_csv("faccao.csv", index=False)

In [45]:
df["faccao_id"] = df.faccao.copy()

In [46]:
df.faccao_id.replace(
    {
        "CV": 1,
        "NEUTRO": 2,
        "NI": 3,
        "PCC": 4,
        "SF": 5,
    },
    regex=True,
    inplace=True,
)

In [47]:
df.head(5)

,Unnamed: 0,nome_completo,vulgo_alcunha,faccao,faccao_funcao,bairro_atual,cidade_atual,uf_atual,area_atuacao,faccao_id
0,1,KAYO NASCIMENTO DE MAGALHÃES,CABRA/ 99,NI,FINANCEIRO,NI,NI,NI,NI,3
1,2,JOSÉ MARIANO DOS SANTOS,RATO,NI,BRAÇO DIREITO,NI,NI,NI,NI,3
2,3,GIVALDO BARBOSA DE FRANÇA,QUINZINHO,NI,CHEFE/LÍDER,NI,NI,NI,NI,3
3,4,JOSÉ ERIVALDO DA PAZ,ÍNDIO,NI,CHEFE/LÍDER,NI,NI,NI,NI,3
4,5,LUCAS MANOEL DA SILVA,PROJETO,CV,SOLDADO DO TRÁFICO,NI,NI,NI,CLIMA BOM/FRONTEIRA,1


## Função na Facção


In [48]:
print(df.groupby("faccao_funcao").size())

faccao_funcao
BRAÇO ARMADO               1
BRAÇO DIREITO              4
CHEFE/LÍDER               56
DISCIPLINA                 1
DISTRIBUIDOR/LOGÍSTICA    12
FINANCEIRO                 3
GERENTE                   37
NI                        12
OUTROS                     1
PAIOL/GUARDA-ROUPAS        4
SOLDADO DO TRÁFICO        74
VENDEDOR                  23
dtype: int64


In [49]:
df_grouped = df.groupby("faccao_funcao").size().reset_index(name="Contagem")
df_grouped = df_grouped.drop("Contagem", axis=1)
df_grouped["funcao_id"] = range(1, len(df_grouped) + 1)  # IDs sequenciais
df_grouped.to_csv("funcao.csv", index=False)

In [50]:
df["funcao_id"] = df.faccao_funcao.copy()

In [51]:
df.funcao_id.replace(
    {
        "BRAÇO ARMADO": 1,
        "BRAÇO DIREITO": 2,
        "CHEFE/LÍDER": 3,
        "DISCIPLINA": 4,
        "DISTRIBUIDOR/LOGÍSTICA": 5,
        "FINANCEIRO": 6,
        "GERENTE": 7,
        "NI": 8,
        "OUTROS": 9,
        "PAIOL/GUARDA-ROUPAS": 10,
        "SOLDADO DO TRÁFICO": 11,
        "VENDEDOR": 12,
    },
    regex=True,
    inplace=True,
)

In [52]:
df.head(15)

,Unnamed: 0,nome_completo,vulgo_alcunha,faccao,faccao_funcao,bairro_atual,cidade_atual,uf_atual,area_atuacao,faccao_id,funcao_id
0,1,KAYO NASCIMENTO DE MAGALHÃES,CABRA/ 99,NI,FINANCEIRO,NI,NI,NI,NI,3,6
1,2,JOSÉ MARIANO DOS SANTOS,RATO,NI,BRAÇO DIREITO,NI,NI,NI,NI,3,2
2,3,GIVALDO BARBOSA DE FRANÇA,QUINZINHO,NI,CHEFE/LÍDER,NI,NI,NI,NI,3,3
3,4,JOSÉ ERIVALDO DA PAZ,ÍNDIO,NI,CHEFE/LÍDER,NI,NI,NI,NI,3,3
4,5,LUCAS MANOEL DA SILVA,PROJETO,CV,SOLDADO DO TRÁFICO,NI,NI,NI,CLIMA BOM/FRONTEIRA,1,11
5,6,VICTOR EMANUEL OLIVEIRA DOS SANTOS,VULGO DA CRUZ,NI,SOLDADO DO TRÁFICO,NI,NI,NI,"CLIMA BOM, ROSANE COLLOR",3,11
6,7,GILVANIO MARCOS OLIVEIRA SANTOS,GIL,NI,GERENTE,NI,NI,NI,GROTA DO CIGANO/ PORTO DE PEDRAS,3,7
7,8,DERIVALDO DA SILVA SANTOS,DERI,NI,CHEFE/LÍDER,NI,NI,NI,GROTA DO CIGANO,3,3
8,9,DARLAN DARLAN PARCEIRA,SCOOBY,PCC,GERENTE,NI,NI,NI,MORRO DO ARI,4,7
9,10,NI,NI,NI,NI,NI,NI,NI,NI,3,8


## Bairros


In [53]:
df.bairro_atual = df.bairro_atual.str.upper()

In [54]:
print(df.groupby("bairro_atual").size())

bairro_atual
ALTO DA BARRA               1
ALTO DE SÃO MARCOS          1
BARRA NOVA                  3
BARRO DURO                  1
BENEDITO BENTES             4
BENEDITO BENTES             2
CABREIRAS                   3
CENTRO                      3
CIDADE UNIVERSITÁRIA        2
COMPLEXO DO ALEMÃO          1
CONJUNTO BARRO PRETO 2      1
DENISSON AMORIM             1
DOM CONSTANTINO             4
ERICK FERRAZ                2
JD BRASÍLIA                 1
JOSÉ DIAS                   1
JOSÉ NUNES                  1
MANOEL TELES                3
MASSAGUEIRA                 1
MASSARANDUBA                1
NI                        174
PARAISÓPOLIS                1
PEDRAS                      2
POEIRA                      1
PONTA GROSSA                1
POÇO                        2
QUILOMBO                    1
SANTA ISABEL                1
SANTA LUCIA                 1
SEM BAIRRO ATUAL            1
TABULEIRO                   1
TAPERAGUÁ                   3
VERGEL DO LAGO.            

In [55]:
df_grouped = df.groupby("bairro_atual").size().reset_index(name="Contagem")
df_grouped = df_grouped.drop("Contagem", axis=1)
df_grouped["bairro_id"] = range(1, len(df_grouped) + 1)  # IDs sequenciais
df_grouped.to_csv("bairro.csv", index=False)

In [56]:
df["bairro_id"] = df.bairro_atual.copy()

In [57]:
df.bairro_id.replace(
    {
        "ALTO DA BARRA": 1,
        "ALTO DE SÃO MARCOS": 2,
        "BARRA NOVA": 3,
        "BARRO DURO": 4,
        "BENEDITO BENTES": 5,
        "BENEDITO BENTES": 6,
        "CABREIRAS": 7,
        "CENTRO": 8,
        "CIDADE UNIVERSITÁRIA": 9,
        "COMPLEXO DO ALEMÃO": 10,
        "CONJUNTO BARRO PRETO": 11,
        "DENISSON AMORIM": 12,
        "DOM CONSTANTINO": 13,
        "ERICK FERRAZ": 14,
        "JD BRASÍLIA": 15,
        "JOSÉ DIAS": 16,
        "JOSÉ NUNES": 17,
        "MANOEL TELES": 18,
        "MASSAGUEIRA": 19,
        "MASSARANDUBA": 20,
        "NI": 21,
        "PARAISÓPOLIS": 22,
        "PEDRAS": 23,
        "POEIRA": 24,
        "PONTA GROSSA": 25,
        "POÇO": 26,
        "QUILOMBO": 27,
        "SANTA ISABEL": 28,
        "SANTA LUCIA": 29,
        "SEM BAIRRO ATUAL": 30,
        "TABULEIRO": 31,
        "TAPERAGUÁ": 32,
        "VERGEL DO LAGO.": 33,
        "ZONA RURAL": 34,
    },
    regex=True,
    inplace=True,
)

## Cidade


In [58]:
df.cidade_atual = df.cidade_atual.str.upper()

In [59]:
print(df.groupby("cidade_atual").size())

cidade_atual
ARAPIRACA                5
ARAPIRACA - AL           1
BARRA DE SÃO MIGUEL      3
CIDADE DE ITAJAÍ         1
CORURIPE                 3
CORURIPE-AL              2
GIRAU DO PONCIANO        1
JOAQUIM GOMES            1
MACEIO                   8
MACEIO                   1
MACEIÓ                   6
MARECHAL DEODORO        21
MARECHALDEODORO          1
NI                     166
PENEDO                   4
PENEDO - AL              1
RIO DE JANEIRO           1
SAO PAULO                1
SÃO PAULO                1
dtype: int64


> Correção dos nomes ou Padronização


In [60]:
df.cidade_atual.replace(
    {
        "ARAPIRACA - AL": "ARAPIRACA",
        "CORURIPE-AL": "CORURIPE",
        "MARECHALDEODORO": "MARECHAL DEODORO",
        "PENEDO - AL": "PENEDO",
        "SAO PAULO": "SÃO PAULO",
        "MACEIO": "MACEIÓ",
    },
    regex=True,
    inplace=True,
)

In [61]:
df_grouped = df.groupby("cidade_atual").size().reset_index(name="Contagem")
df_grouped = df_grouped.drop("Contagem", axis=1)
df_grouped["cidade_id"] = range(1, len(df_grouped) + 1)  # IDs sequenciais
df_grouped.to_csv("cidade.csv", index=False)

In [62]:
df["cidade_id"] = df.cidade_atual.copy()

In [63]:
df.cidade_id.replace(
    {
        "ARAPIRACA": 1,
        "BARRA DE SÃO MIGUEL": 2,
        "CIDADE DE ITAJAÍ": 3,
        "CORURIPE": 4,
        "GIRAU DO PONCIANO": 5,
        "JOAQUIM GOMES": 6,
        "MACEIÓ": 7,
        "MACEIÓ": 8,
        "MARECHAL DEODORO": 9,
        "NI": 10,
        "PENEDO": 11,
        "RIO DE JANEIRO": 12,
        "SÃO PAULO": 13,
    },
    regex=True,
    inplace=True,
)

In [64]:
df.head()

,Unnamed: 0,nome_completo,vulgo_alcunha,faccao,faccao_funcao,bairro_atual,cidade_atual,uf_atual,area_atuacao,faccao_id,funcao_id,bairro_id,cidade_id
0,1,KAYO NASCIMENTO DE MAGALHÃES,CABRA/ 99,NI,FINANCEIRO,NI,NI,NI,NI,3,6,21,10
1,2,JOSÉ MARIANO DOS SANTOS,RATO,NI,BRAÇO DIREITO,NI,NI,NI,NI,3,2,21,10
2,3,GIVALDO BARBOSA DE FRANÇA,QUINZINHO,NI,CHEFE/LÍDER,NI,NI,NI,NI,3,3,21,10
3,4,JOSÉ ERIVALDO DA PAZ,ÍNDIO,NI,CHEFE/LÍDER,NI,NI,NI,NI,3,3,21,10
4,5,LUCAS MANOEL DA SILVA,PROJETO,CV,SOLDADO DO TRÁFICO,NI,NI,NI,CLIMA BOM/FRONTEIRA,1,11,21,10


## UF atual


In [65]:
df.uf_atual = df.uf_atual.str.upper()

In [66]:
print(df.groupby("uf_atual").size())

uf_atual
AL     56
NI    167
PE      1
RJ      1
SC      1
SP      2
dtype: int64


In [67]:
df_grouped = df.groupby("uf_atual").size().reset_index(name="Contagem")
df_grouped = df_grouped.drop("Contagem", axis=1)
df_grouped["uf_id"] = range(1, len(df_grouped) + 1)  # IDs sequenciais
df_grouped.to_csv("uf.csv", index=False)

In [68]:
df["uf_id"] = df.uf_atual.copy()

In [69]:
df.uf_id.replace(
    {
        "AL": 1,
        "NI": 2,
        "PE": 3,
        "RJ": 4,
        "SC": 5,
        "SP": 6,
    },
    regex=True,
    inplace=True,
)

In [70]:
df.head()

,Unnamed: 0,nome_completo,vulgo_alcunha,faccao,faccao_funcao,bairro_atual,cidade_atual,uf_atual,area_atuacao,faccao_id,funcao_id,bairro_id,cidade_id,uf_id
0,1,KAYO NASCIMENTO DE MAGALHÃES,CABRA/ 99,NI,FINANCEIRO,NI,NI,NI,NI,3,6,21,10,2
1,2,JOSÉ MARIANO DOS SANTOS,RATO,NI,BRAÇO DIREITO,NI,NI,NI,NI,3,2,21,10,2
2,3,GIVALDO BARBOSA DE FRANÇA,QUINZINHO,NI,CHEFE/LÍDER,NI,NI,NI,NI,3,3,21,10,2
3,4,JOSÉ ERIVALDO DA PAZ,ÍNDIO,NI,CHEFE/LÍDER,NI,NI,NI,NI,3,3,21,10,2
4,5,LUCAS MANOEL DA SILVA,PROJETO,CV,SOLDADO DO TRÁFICO,NI,NI,NI,CLIMA BOM/FRONTEIRA,1,11,21,10,2


In [74]:
df = df.drop('Unnamed: 0', axis=1)
df.to_csv("faccionado.csv", index=False)

In [75]:
print(list(df.columns))
len(df.columns)

['nome_completo', 'vulgo_alcunha', 'faccao', 'faccao_funcao', 'bairro_atual', 'cidade_atual', 'uf_atual', 'area_atuacao', 'faccao_id', 'funcao_id', 'bairro_id', 'cidade_id', 'uf_id']


13

## Área de Atuação


In [ ]:
df.area_atuacao = df.area_atuacao.str.upper()

In [ ]:
print(df.groupby("area_atuacao").size())

area_atuacao
 CLIMA BOM/FRONTEIRA, CONJUNTO JARBAS OITICICA, CONJUNTO ANTÔNIO LINS (RIO LARGO) E LOTEAMENTO PALMAR NO RIO NOVO                                       1
11º BPM                                                                                                                                                 9
1A PONTE VALE DO REGINALDO                                                                                                                              1
9ª CPM/I - CORURIPE-AL                                                                                                                                  1
ALAMEDAS DA PAJUÇARA                                                                                                                                    2
ALAMEDAS PAJUÇARA                                                                                                                                       1
ALTO DO SÃO MIGUEL, RIO LARGO                                  

In [ ]:
# df_grouped = df.groupby("area_atuacao").size().reset_index(name="Contagem")
# df_grouped = df_grouped.drop("Contagem", axis=1)
# df_grouped["area_atuacao_id"] = range(1, len(df_grouped) + 1)  # IDs sequenciais
# df_grouped.to_csv("carea_atuacao.csv", index=False)

In [ ]:
# df["area_atuacao_id"] = df.area_atuacao.copy()

# df.area_atuacao_id.replace(
#     {
#         "ARAPIRACA": 1,
#         "BARRA DE SÃO MIGUEL": 2,
#         "CIDADE DE ITAJAÍ": 3,
#         "CORURIPE": 4,
#         "GIRAU DO PONCIANO": 5,
#         "JOAQUIM GOMES": 6,
#         "MACEIÓ": 7,
#         "MACEIÓ": 8,
#         "MARECHAL DEODORO": 9,
#         "NI": 10,
#         "PENEDO": 11,
#         "RIO DE JANEIRO": 12,
#         "SÃO PAULO": 13,
#     },
#     regex=True,
#     inplace=True,
# )

# df.head()